<a href="https://colab.research.google.com/github/techwithanirudh/Automatic-Image-Captioning/blob/master/SignsClassifiction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"superkaggleuserinjan","key":"d8cc974e68b63884566d5cb02a7ea3bb"}'}

In [ ]:
!pip install -q kaggle

     |████████████████████████████████| 61kB 4.8MB/s 


In [ ]:
# create folder called kaggle
!mkdir -p ~/.kaggle
# copy kaggle.json into kaggle folder
!cp /content/kaggle.json ~/.kaggle/
# change permissions so that only you can read-write
!chmod 600 /root/.kaggle/kaggle.json

'chmod 600 /root/.kaggle/kaggle.json'

In [ ]:
import os

print(os.listdir('/root/'))
print(os.listdir('/root/.kaggle/'))

with open('/root/.kaggle/kaggle.json') as f:
  print(f.read())

print(os.listdir('/content/'))

['.bashrc', '.profile', '.cache', '.keras', '.kaggle', '.local', '.tmux.conf', '.jupyter', '.config', '.ipython', '.npm', '.gsutil']
['kaggle.json']
{"username":"superkaggleuserinjan","key":"d8cc974e68b63884566d5cb02a7ea3bb"}
['.config', 'kaggle.json', 'gtsrb-german-traffic-sign.zip', '.ipynb_checkpoints', 'sample_data']


In [ ]:
!kaggle datasets download -d meowmeowmeowmeowmeow/gtsrb-german-traffic-sign

 97% 594M/612M [00:01<00:00, 345MB/s]
100% 612M/612M [00:01<00:00, 324MB/s]


In [ ]:
!unzip /content/gtsrb-german-traffic-sign.zip

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import zipfile
with zipfile.ZipFile('/content/drive/MyDrive/Python-Project-Traffic-Sign-Classification.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/Python-Project-Traffic-Sign-Classification')

In [62]:
!cp /content/test -r "/content/Python-Project-Traffic-Sign-Classification/Traffic sign classification/test"
!cp /content/train -r "/content/Python-Project-Traffic-Sign-Classification/Traffic sign classification/train"
!cp /content/meta -r "/content/Python-Project-Traffic-Sign-Classification/Traffic sign classification/meta"

Main -> (DONT RUN THIS) -> (Because this is training it may take many hours so used tge trained model)

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from PIL import Image
import os
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout

data = []
labels = []
classes = 43
cur_path = os.getcwd()

for i in range(classes):
    path = os.path.join(cur_path,'train',str(i))
    images = os.listdir(path)

    for a in images:
        try:
            image = Image.open(path + '\\'+ a)
            image = image.resize((30,30))
            image = np.array(image)
            #sim = Image.fromarray(image)
            data.append(image)
            labels.append(i)
        except:
            print("Error loading image")

data = np.array(data)
labels = np.array(labels)

In [ ]:
print(data.shape, labels.shape)
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

y_train = to_categorical(y_train, 43)
y_test = to_categorical(y_test, 43)

In [ ]:
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=X_train.shape[1:]))
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(43, activation='softmax'))

#Compilation of the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
epochs = 15
history = model.fit(X_train, y_train, batch_size=32, epochs=epochs, validation_data=(X_test, y_test))
model.save("my_model.h5")

In [ ]:
plt.figure(0)
plt.plot(history.history['accuracy'], label='training accuracy')
plt.plot(history.history['val_accuracy'], label='val accuracy')
plt.title('Accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show()

plt.figure(1)
plt.plot(history.history['loss'], label='training loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.title('Loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

In [68]:
#testing accuracy on test dataset
from sklearn.metrics import accuracy_score

y_test = pd.read_csv('Test.csv')

labels = y_test["ClassId"].values
imgs = y_test["Path"].values

data=[]

for img in imgs:
    image = Image.open(img)
    image = image.resize((30,30))
    data.append(np.array(image))

X_test=np.array(data)

pred = model.predict_classes(X_test)

#Accuracy with the test data
from sklearn.metrics import accuracy_score
print(accuracy_score(labels, pred))

model.save('traffic_classifier.h5')

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


0.966904196357878
0.966904196357878


<- Main

Testing ->

In [ ]:
# testing accuracy on test dataset
from sklearn.metrics import accuracy_score
import pandas as pd
from PIL import Image
import numpy as np
from keras.models import load_model
model = load_model('/content/Python-Project-Traffic-Sign-Classification/Traffic sign classification/traffic_classifier.h5')

y_test = pd.read_csv('Test.csv')

labels = y_test["ClassId"].values
imgs = y_test["Path"].values

data=[]

for img in imgs:
    image = Image.open(img)
    image = image.resize((30,30))
    data.append(np.array(image))

X_test=np.array(data)

pred = model.predict_classes(X_test)

#Accuracy with the test data
from sklearn.metrics import accuracy_score
print(accuracy_score(labels, pred))

# model.save('traffic_classifier.h5')

#Accuracy with the test data
from sklearn.metrics import accuracy_score
print(accuracy_score(labels, pred))

Test is Done Now This Is Fun

In [74]:
# import os
# os.chdir(os.path.dirname(__file__))
from google.colab import files
from PIL import Image
import numpy
#load the trained model to classify sign
from keras.models import load_model
model = load_model('/content/Python-Project-Traffic-Sign-Classification/Traffic sign classification/traffic_classifier.h5')

#dictionary to label all traffic signs class.
classes = { 1:'Speed limit (20km/h)',
            2:'Speed limit (30km/h)',      
            3:'Speed limit (50km/h)',       
            4:'Speed limit (60km/h)',      
            5:'Speed limit (70km/h)',    
            6:'Speed limit (80km/h)',      
            7:'End of speed limit (80km/h)',     
            8:'Speed limit (100km/h)',    
            9:'Speed limit (120km/h)',     
           10:'No passing',   
           11:'No passing veh over 3.5 tons',     
           12:'Right-of-way at intersection',     
           13:'Priority road',    
           14:'Yield',     
           15:'Stop',       
           16:'No vehicles',       
           17:'Veh > 3.5 tons prohibited',       
           18:'No entry',       
           19:'General caution',     
           20:'Dangerous curve left',      
           21:'Dangerous curve right',   
           22:'Double curve',      
           23:'Bumpy road',     
           24:'Slippery road',       
           25:'Road narrows on the right',  
           26:'Road work',    
           27:'Traffic signals',      
           28:'Pedestrians',     
           29:'Children crossing',     
           30:'Bicycles crossing',       
           31:'Beware of ice/snow',
           32:'Wild animals crossing',      
           33:'End speed + passing limits',      
           34:'Turn right ahead',     
           35:'Turn left ahead',       
           36:'Ahead only',      
           37:'Go straight or right',      
           38:'Go straight or left',      
           39:'Keep right',     
           40:'Keep left',      
           41:'Roundabout mandatory',     
           42:'End of no passing',      
           43:'End no passing veh > 3.5 tons' }

# uploaded = files.upload()
# file_path = uploaded.keys()[0]
file_path = '/content/Python-Project-Traffic-Sign-Classification/Traffic sign classification/train/0/00000_00000_00000.png'
image = Image.open(file_path)
image = image.resize((30,30))
image = numpy.expand_dims(image, axis=0)
image = numpy.array(image)
print(image.shape)
pred = model.predict_classes([image])[0]
sign = classes[pred+1]
print('Sign:', sign)

(1, 30, 30, 3)
Sign: Speed limit (20km/h)


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


<- Testing 

In [ ]:
import os, shutil

files = os.listdir()
ignoredFiles = ['.ipynb_checkpoints', 'sample_data', '.config', 'kaggle.json']

# print(files)

for file in files:
  index = 0
  indexL = []
  for ignoredFile in ignoredFiles:
    index += 1
    if file != ignoredFile:
      indexL.append(index)
  if len(indexL) == 4:
    print(file)
    try:
      shutil.rmtree(file)
    except:
      try:
        os.remove('/content/' + file)
      except Exception as e:
        print('Error: ', e)